In [4]:
from nytsnippetgetter import get_data
import time

# start timer
start_time = time.time()

# get available number of pages for each topic. Each page is equivalent to 10 articles
topics=['economics','politics','espionage','global+warming', 'clinton', 'sanders', 'guns', 'cancer', 'sex']
npages = [1500,1000,500,100,100,100,100,100,100]
# topics=['economics','politics']
get_data(topics, BEGINDATE = 20131213, LIMITS=True) # articles written since 2013-December-13

Total number of pages available (each page is 10 articles): 
economics :  4564
politics :  19069
espionage :  1173
global+warming :  2022
clinton :  9168
sanders :  3497
guns :  3356
cancer :  7599
sex :  9963


In [5]:
# download article data 
articles = get_data(TOPICS = topics, NPAGES = npages, BEGINDATE = 20131213)
# articles = get_data(TOPICS = topics, NPAGES = [150,100,50,10,10,10,10,10,10])
# articles = get_data(TOPICS = topics, BEGINDATE = 20131213, NPAGES = [15,10])

Topics:  ['economics', 'politics', 'espionage', 'global+warming', 'clinton', 'sanders', 'guns', 'cancer', 'sex']
NPages:  [1500, 1000, 500, 100, 100, 100, 100, 100, 100] 

Total documents:  36000
Started download...
economics is done | 1500/3600
politics is done | 2500/3600
espionage is done | 3000/3600
global+warming is done | 3100/3600
clinton is done | 3200/3600
sanders is done | 3300/3600
guns is done | 3400/3600
cancer is done | 3500/3600
sex is done | 3600/3600

Done in  390.8875939846039 seconds


In [6]:
# articles is a list of objects with each object being one document
articles[0]

{'abstract': 'Psychology in economics',
 'author': '',
 'date_modified': '2008-11-03T23:52:36Z',
 'date_published': '1921-04-10T00:00:00Z',
 'keywords': [{'name': 'persons', 'value': 'ECONOMICS'},
  {'name': 'glocations', 'value': 'RUSSIA'},
  {'name': 'glocations', 'value': 'ENGLAND'},
  {'name': 'organizations', 'value': 'INTERNATL ARMY'},
  {'name': 'subject', 'value': 'BOLSHEVIST GOVT'},
  {'name': 'subject', 'value': 'LABOR'},
  {'name': 'subject', 'value': 'EDITORIALS'}],
 'lead_paragraph': None,
 'nytclass': None,
 'section_name': [],
 'snippet': 'Psychology in economics',
 'title': 'REDS FORMING ARMY OF ALL NATIONS; Said to Plan a Huge Force to Enforce Dictates of the ThirdInternationale.',
 'user_topic': 'economics',
 'weburl': 'http://query.nytimes.com/gst/abstract.html?res=9901E0D9103FEE3ABC4852DFB266838A639EDE'}

In [7]:
# number of articles
len(articles)

36000

In [8]:
# some articles are missing lead paragraph and abstract. just use snippets for now
data = [ x['snippet'] for x in articles]

# check if data has Nones
nonesidx = [data.index(x) for x in data if x == None and len(x) > 0]
if len(nonesidx) > 0:
    print("You have nones. Below are indices of articles.")
    print(nonesidx)
else:
    print("No Nones. Good to go.")
    
data[0:3]

No Nones. Good to go.


['Psychology in economics',
 'GASOLINE prices on the South Fork are consistently 20 to 50 cents more per gallon than in other areas of Long Island. But the recent run-up in gasoline prices to record levels has been a rude awakening for some South Fork residents.',
 'we have to make these comparisons, and we either do them implicitly or explicitly,” said Dana Goldman, director of health <strong>economics</strong> at the RAND Corporation, a nonprofit research institute in Santa Monica, Calif. To make the']

In [17]:
# article topics
label = [ x['user_topic'] for x in articles]
label[-5:-1]

['sex', 'sex', 'sex', 'sex']

In [19]:
# http://scikit-learn.org/stable/datasets/twenty_newsgroups.html

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

vectorizer = TfidfVectorizer() # Convert a collection of raw documents to a matrix of TF-IDF features.
le = preprocessing.LabelEncoder() # Encode labels with value between 0 and n_classes-1.

vectors = vectorizer.fit_transform(data) 
target = le.fit_transform(label)

print(vectors.shape, target.shape,'\n')
print("Number of unique words (features) across all docs: ", vectors.shape[1], '\n')
print("(docid, wordid) TFIDF", '\n')
print(vectors[0,])

(36000, 1267) (36000,) 

Number of unique words (features) across all docs:  1267 

(docid, wordid) TFIDF 

  (0, 394)	0.404664224029
  (0, 571)	0.328499578152
  (0, 900)	0.853425388036


In [20]:
# average number of non-zero components by sample, i.e. average number of words with non-zero IDFxTF by article
vectors.nnz / float(vectors.shape[0])

24.13888888888889

In [21]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(vectors, target, test_size=0.33, random_state=1729)
print(X_train.shape, X_test.shape)

(24120, 1267) (11880, 1267)


In [22]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

import pandas as pd

clf = MultinomialNB(alpha=2)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print("F-score: ", metrics.f1_score(y_test, pred, average='weighted'))
print("Accuracy: ", metrics.precision_score(y_test, pred, average='weighted'))
print("Confusion matrix:\n", pd.crosstab(y_test, pred, rownames=['True'], colnames=['Predicted']))

F-score:  1.0
Accuracy:  1.0
Confusion matrix:
 Predicted    0    1     2     3    4    5     6    7    8
True                                                     
0          337    0     0     0    0    0     0    0    0
1            0  321     0     0    0    0     0    0    0
2            0    0  4910     0    0    0     0    0    0
3            0    0     0  1670    0    0     0    0    0
4            0    0     0     0  334    0     0    0    0
5            0    0     0     0    0  343     0    0    0
6            0    0     0     0    0    0  3327    0    0
7            0    0     0     0    0    0     0  313    0
8            0    0     0     0    0    0     0    0  325


In [23]:
import numpy as np

def show_top10(classifier, vectorizer, categories):
    feature_names = np.asarray(vectorizer.get_feature_names())
    for i, category in enumerate(categories):
        top10 = np.argsort(classifier.coef_[i])[-10:]
        print("%s: %s" % (category, " ".join(feature_names[top10])))
        
# print ten most important words for each topic
show_top10(clf, vectorizer, le.classes_)

cancer: immunotherapy of breast and the her as for strong cancer
clinton: with and trump art in the hillary hill strong clinton
economics: he and that of to psychology in the economics strong
espionage: by in and new had his to strong espionage the
global+warming: has to group planet the climate strong warming change global
guns: night of fort increase at roses for strong the guns
politics: to by may province havana oriente arms washington of the
sanders: host weight 59 woman bernie show larry strong the sanders
sex: production partly of its strangers in and the strong sex


In [24]:
# end timer
print(time.time()-start_time, 'seconds')

1651.8414900302887 seconds


In [25]:
# test the model on articles written before 2013-December-13
test = get_data(TOPICS = topics, ENDDATE = 20131213, NPAGES = [10,10,10,10,10,10,10,10,10,10])
test_data = [ x['snippet'] for x in test]
test_label = [ x['user_topic'] for x in test]

test_vectors = vectorizer.transform(test_data) 
test_target = le.transform(test_label)

print(test_vectors.shape, test_target.shape,'\n')
print("Number of unique words (features) across all docs: ", vectors.shape[1], '\n')
print("(docid, wordid) TFIDF", '\n')
print(vectors[0,])

Topics:  ['economics', 'politics', 'espionage', 'global+warming', 'clinton', 'sanders', 'guns', 'cancer', 'sex']
NPages:  [10, 10, 10, 10, 10, 10, 10, 10, 10, 10] 

Total documents:  1000
Started download...
economics is done | 10/100
politics is done | 20/100
espionage is done | 30/100
global+warming is done | 40/100
clinton is done | 50/100
sanders is done | 60/100
guns is done | 70/100
cancer is done | 80/100
sex is done | 90/100

Done in  26.307778120040894 seconds
(900, 1267) (900,) 

Number of unique words (features) across all docs:  1267 

(docid, wordid) TFIDF 

  (0, 394)	0.404664224029
  (0, 571)	0.328499578152
  (0, 900)	0.853425388036


In [26]:
pred_test = clf.predict(test_vectors)
print("F-score: ", metrics.f1_score(test_target, pred_test, average='weighted'))
print("Accuracy: ", metrics.precision_score(test_target, pred_test, average='weighted'))
print("Confusion matrix:\n", pd.crosstab(test_target, pred_test, rownames=['True'], colnames=['Predicted']))

F-score:  1.0
Accuracy:  1.0
Confusion matrix:
 Predicted    0    1    2    3    4    5    6    7    8
True                                                  
0          100    0    0    0    0    0    0    0    0
1            0  100    0    0    0    0    0    0    0
2            0    0  100    0    0    0    0    0    0
3            0    0    0  100    0    0    0    0    0
4            0    0    0    0  100    0    0    0    0
5            0    0    0    0    0  100    0    0    0
6            0    0    0    0    0    0  100    0    0
7            0    0    0    0    0    0    0  100    0
8            0    0    0    0    0    0    0    0  100


# Classify by author

In [27]:
# article author
label = [ x['author'] for x in articles]
label[-5:-1]

['DWIGHT GARNER', 'BEN KENIGSBERG', '', 'LESLIE BERGER']

In [28]:
target = le.fit_transform(label)

X_train, X_test, y_train, y_test = train_test_split(vectors, target, test_size=0.33, random_state=1729)
print(X_train.shape, X_test.shape)

(24120, 1267) (11880, 1267)


In [29]:
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print("F-score: ", metrics.f1_score(y_test, pred, average='weighted'))
print("Accuracy: ", metrics.precision_score(y_test, pred, average='weighted'))
print("Confusion matrix:\n", pd.crosstab(y_test, pred, rownames=['True'], colnames=['Predicted']))

F-score:  1.0
Accuracy:  1.0
Confusion matrix:
 Predicted    0    1   2   3   4   5   6    7   8    9  ...  49   50   51  52  \
True                                                   ...                     
0          4507    0   0   0   0   0   0    0   0    0 ...   0    0    0   0   
1             0  482   0   0   0   0   0    0   0    0 ...   0    0    0   0   
2             0    0  39   0   0   0   0    0   0    0 ...   0    0    0   0   
3             0    0   0  65   0   0   0    0   0    0 ...   0    0    0   0   
4             0    0   0   0  38   0   0    0   0    0 ...   0    0    0   0   
5             0    0   0   0   0  35   0    0   0    0 ...   0    0    0   0   
6             0    0   0   0   0   0  34    0   0    0 ...   0    0    0   0   
7             0    0   0   0   0   0   0  190   0    0 ...   0    0    0   0   
8             0    0   0   0   0   0   0    0  26    0 ...   0    0    0   0   
9             0    0   0   0   0   0   0    0   0  171 ...   0    0    0

In [30]:
# print ten most important words for each topic
show_top10(clf, vectorizer, le.classes_)

: washington strong exposure for and economics of in the psychology
ALEX BERENSON: either monica corporation goldman them calif comparisons rand we make
AMY CHOZICK: victories bearing delegates thirds far leads string direction more sanders
AMY HARMON: treatment approach months toxic championed promising results standard for cancer
ANDREW R. CHOW: guitarist axl duties frontman angus will guns roses ac dc
ANNIE CORREAL: son previously 21 alpert month endings spends defender obit kalyan
BEN KENIGSBERG: 91 wfmu hudson radio hardly film station 90 as city
BENJAMIN WEISER: americans sporyshev buryakov prisoners based television four the he mr
BRYAN MILLER: really vibrant sights always williamsburg lefferts willing clinton hill she
CEYLAN YEGINSU: closed potential journalists prompting awaited whether istanbul outrage concerns trial
CHARLES ISHERWOOD: thomas lucas season timing du hnath top trust continuing here
CHARLES M. BLOW: bash thursday planned cnn unsettling comment fight me if democr

# Classify by author using articles from different time periods

In [31]:
# Classify by author using articles from different time periods
articles = get_data(TOPICS = topics, NPAGES = [150,100,50,10,10,10,10,10,10])
print(articles[0])
# number of articles
print(len(articles))

Topics:  ['economics', 'politics', 'espionage', 'global+warming', 'clinton', 'sanders', 'guns', 'cancer', 'sex']
NPages:  [150, 100, 50, 10, 10, 10, 10, 10, 10] 

Total documents:  3600
Started download...
economics is done | 150/360
politics is done | 250/360
espionage is done | 300/360
global+warming is done | 310/360
clinton is done | 320/360
sanders is done | 330/360
guns is done | 340/360
cancer is done | 350/360
sex is done | 360/360

Done in  47.70070290565491 seconds
{'user_topic': 'economics', 'abstract': 'Psychology in economics', 'weburl': 'http://query.nytimes.com/gst/abstract.html?res=9901E0D9103FEE3ABC4852DFB266838A639EDE', 'author': '', 'nytclass': None, 'date_modified': '2008-11-03T23:52:36Z', 'keywords': [{'name': 'persons', 'value': 'ECONOMICS'}, {'name': 'glocations', 'value': 'RUSSIA'}, {'name': 'glocations', 'value': 'ENGLAND'}, {'name': 'organizations', 'value': 'INTERNATL ARMY'}, {'name': 'subject', 'value': 'BOLSHEVIST GOVT'}, {'name': 'subject', 'value': 'LABOR

In [32]:
# some articles are missing lead paragraph and abstract. just use snippets for now
data = [ x['snippet'] for x in articles]

# check if data has Nones
nonesidx = [data.index(x) for x in data if x == None and len(x) > 0]
if len(nonesidx) > 0:
    print("You have nones. Below are indices of articles.")
    print(nonesidx)
else:
    print("No Nones. Good to go.")
    
data[0:3]

No Nones. Good to go.


['Psychology in economics',
 'GASOLINE prices on the South Fork are consistently 20 to 50 cents more per gallon than in other areas of Long Island. But the recent run-up in gasoline prices to record levels has been a rude awakening for some South Fork residents.',
 'we have to make these comparisons, and we either do them implicitly or explicitly,” said Dana Goldman, director of health <strong>economics</strong> at the RAND Corporation, a nonprofit research institute in Santa Monica, Calif. To make the']

In [33]:
# article authors
label = [ x['author'] for x in articles]
label[-5:-1]

['DWIGHT GARNER', 'LAURIE GOODSTEIN', 'BEN KENIGSBERG', '']

In [35]:
vectors = vectorizer.fit_transform(data)
target = le.fit_transform(label)

X_train, X_test, y_train, y_test = train_test_split(vectors, target, test_size=0.33, random_state=1729)
print(X_train.shape, X_test.shape)

(2412, 1251) (1188, 1251)


In [36]:
clf.fit(X_train, y_train)
pred = clf.predict(X_test)
print("F-score: ", metrics.f1_score(y_test, pred, average='weighted'))
print("Accuracy: ", metrics.precision_score(y_test, pred, average='weighted'))
print("Confusion matrix:\n", pd.crosstab(y_test, pred, rownames=['True'], colnames=['Predicted']))

F-score:  0.834503858119
Accuracy:  0.800488961534
Confusion matrix:
 Predicted   0   1   2   4   6   7   8   10  11  13 ...  39  40  42  44  47  \
True                                               ...                       
0          453   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   
1            0  48   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   
2            4   0   1   0   0   0   0   0   0   0 ...   0   0   0   0   0   
3            4   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   
4            0   0   0   2   0   0   0   0   0   0 ...   0   0   0   0   0   
6            0   0   0   0  17   0   0   0   0   0 ...   0   0   0   0   0   
7            0   0   0   0   0   2   0   0   0   0 ...   0   0   0   0   0   
8            0   0   0   0   0   0  14   0   0   0 ...   0   0   0   0   0   
9            0   0   0   0   0   0   0   0   0   0 ...   0   0   0   0   0   
10           0   0   0   0   0   0   0   1   0   0 ...   0   0   0   0  

//anaconda/envs/tpot/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
//anaconda/envs/tpot/lib/python3.5/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [41]:
print("Homogeneity: %0.3f" % metrics.homogeneity_score(y_test, pred))
print("Completeness: %0.3f" % metrics.completeness_score(y_test, pred))
print("V-measure: %0.3f" % metrics.v_measure_score(y_test, pred))
print("Adjusted Rand-Index: %.3f"
      % metrics.adjusted_rand_score(y_test, pred))
print("Silhouette Coefficient: %0.3f"
      % metrics.silhouette_score(X_test, pred, sample_size=1000))

Homogeneity: 0.801
Completeness: 0.991
V-measure: 0.886
Adjusted Rand-Index: 0.796
Silhouette Coefficient: 0.458
